## BentoML 연동  

In [1]:
import bentoml
from ultralytics import YOLO

In [2]:
model = YOLO("yolov8x.pt").model
#model.eval()

In [3]:
metadata = {  
    'acc': 0.9,
    'dataset_version': '20240808',
}

#signature = {'predict': {'batchable': False}}  # model 코드 내 predict에서 호출
signature = {"__call__": {"batchable": False}}  # model 코드에서 forward로 직접 호출

In [4]:
saved_model = bentoml.pytorch.save_model(
    'yolov8x',
    model,
    signatures=signature,
    metadata=metadata, 
)

print(saved_model)

Model(tag="yolov8x:cl77uwkvgkkhxqb4")


In [ ]:
model = YOLO("yolov8m.pt").model
model.eval()
saved_model = bentoml.pytorch.save_model(name='yolov8m_model',
                                         model=model,
                                         signatures={"__call__": {"batchable": False}})

In [3]:
class PoopResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet18(weights='ResNet18_Weights.DEFAULT')

        # # Freeze weights
        # for param in model_ft.parameters():
        #     param.requires_grad = False

        self.model.fc = nn.Sequential(
            nn.Linear(self.model.fc.in_features, 128),
            nn.ReLU(),
            nn.Dropout(.3),
            nn.Linear(128,3),
            nn.LogSoftmax(dim=1)
        )
        
        self.model.load_state_dict(torch.load('poop_resnet18_best0928.pt'))
        self.softmax = nn.Softmax(dim=1)
        
#         self.xforms = transforms.Compose([
#                 transforms.Resize(256),
#                 transforms.CenterCrop(224),
#                 transforms.ToTensor(),
#                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#             ])
        

    def forward(self, x):
        # x = self.xforms(x)
        return self.model(x)

    def predict(self, inp):
        self.eval()
        with torch.no_grad():
            output = self(inp)
            output = self.softmax(output)
            # _, pred = torch.max(output, 1)
            return output
        
        
poopup = PoopResNet()

In [7]:
metadata = {  
    'acc': 0.928,
    'dataset_version': '20230306',
}

signature = {'predict': {'batchable': True}}  # 호출되는 방법을 정의

In [8]:
saved_model = bentoml.pytorch.save_model(
    'poopup',
    poopup,
    signatures=signature,
    metadata=metadata, 
)

print(saved_model)

Model(tag="poopup:j6c5vuv3zskmxqb4")


In [9]:
!bentoml models list
# !bentoml models delete poopup:fttpe7fueke3zqb4

 Tag                           Module           Size       Creation Time       
 poopup:j6c5vuv3zskmxqb4       bentoml.pytorch  42.97 MiB  2023-03-06 12:09:30 
 poopup:5btv7hf3zoln5qb4       bentoml.pytorch  42.97 MiB  2023-03-06 12:06:37 
 iris_clf:t23z5cvwigmppqb4     bentoml.sklearn  5.98 KiB   2023-02-27 10:54:07 
 poopup:fttpe7fueke3zqb4       bentoml.pytorch  42.98 MiB  2023-02-24 18:03:59 
 pytorch_mnist:jtn5nvnrygazt���  bentoml.pytorch  1.30 MiB   2023-02-21 17:25:29 
 poop_resnet18_best0928:x7nf���  bentoml.pytorch  42.97 MiB  2023-02-21 11:37:57 


In [17]:
bento_model: bentoml.Model = bentoml.models.get('poopup:latest')
# bento_model: bentoml.Model = bentoml.models.get('poopup:fttpe7fueke3zqb4')
    
print(bento_model.tag)
print(bento_model.path)
print(bento_model.custom_objects)
print(bento_model.info.metadata)
print(bento_model.info.labels)

my_runner: bentoml.Runner = bento_model.to_runner()
my_runner.init_local()
print(my_runner)

'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.


poopup:j6c5vuv3zskmxqb4
C:\Users\user\bentoml\models\poopup\j6c5vuv3zskmxqb4
{}
{'acc': 0.928, 'dataset_version': '20230306'}
{}
Runner(name='poopup', models=[Model(tag="poopup:j6c5vuv3zskmxqb4", path="C:\Users\user\bentoml\models\poopup\j6c5vuv3zskmxqb4")], resource_config=None, runnable_class=<class 'bentoml._internal.frameworks.common.pytorch.partial_class.<locals>.NewClass'>, runner_methods=[RunnerMethod(runner=..., name='predict', config=RunnableMethodConfig(batchable=True, batch_dim=(0, 0), input_spec=None, output_spec=None), max_batch_size=100, max_latency_ms=10000)], scheduling_strategy=<class 'bentoml._internal.runner.strategy.DefaultStrategy'>, runnable_init_params={}, _runner_handle=<bentoml._internal.runner.runner_handle.local.LocalRunnerRef object at 0x000001E6835DBA30>)


In [18]:
xforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

img = Image.open('dog.jpg')
img = np.expand_dims(xforms(img), 0)

img.shape

(1, 3, 224, 224)

In [16]:
my_runner.run(img)  # without softmax

tensor([[-0.9025, -1.2581, -1.1703]])

In [19]:
my_runner.run(img)

tensor([[0.4055, 0.2842, 0.3103]])

In [20]:
torch.cuda.empty_cache()